# Vorhersage Borkenkäferbefall ML
## Entwicklung eines Modells zur Abschätzung des für vorgegebene räumliche Verwaltungseinheiten in Sachsen für unterschiedliche Zeitintervalle zu erwartenden Buchdruckerbefalls auf Grundlage der Daten der Befallsentwicklung und zum Witterungsverlauf bis zum Vorhersagezeitpunkt sowie von maximal drei Witterungsszenarien für den Vorhersagezeitraum.   

**Author**
Yannic Holländer

In diesem Notebook wird das im Rahmen des o.g. Projekts erstellte Machine Learning Modell verwendet, um für drei durch den Benutzer definierbare Wetterszenarien Vorhersagen des durch den Buchdrucker verursachten Schadholzes für ein bestimmtes Zeitintervall zu treffen. 

Das Notebook ist für die Benutzung in Google Colab angepasst. Der Code kann in Colab in seiner Gesamtheit ausgeführt werden mit Klick auf "Laufzeit" -> "Alle ausführen" in der Menüleiste. Alternativ können einzelne Codeblöcke mit Klick auf den "Zelle ausführen" Button oder, wenn der Codeblock aktiv ist, mit STRG+Enter ausgeführt werden. Wichtig ist, dass nach dem Ausführen des ersten Codeblocks im Notebook über den Dialog die zwei Dateien "input.xlsx" sowie "model.pkl" hochgeladen werden, da diese für die Vorhersage benötigt werden. Die Datei "input.xlsx" enthält alle bisherigen Beobachtungen in einer Excel-Tabelle. Diese sollte mit den realen Beobachtungen ergänzt werden, sobald diese vollständig für einen Zeitraum vorliegen. Da vergangene Niederschläge und Zugänge von Schadholz benötigt werden, können ansonsten keine akkuraten Vorhersagen getroffen werden. Die Datei "model.pkl" enthält das Vorhersagemodell in der im Rahmen des Projekts erstellten Form.  

Sobald die Dateien einmal hochgeladen wurden, sind sie auch im Dateibrowser (Datei-Symbol in der linken Menüleiste) zu sehen und verbleiben dort, bis die Laufzeit auf Werkseinstellungen zurückgesetzt wurde.

Wird das Notebook lokal auf dem Rechner und nicht in Google Colab genutzt, werden der erste und letzte Codeblock zum hoch- und herunterladen von Dateien nicht benötigt. In diesem Fall können sie daher gelöscht oder auskommentiert werden. Die benötigten Dateien sollten dann stattdessen im selben Ordner wie das Notebook vorliegen, die Vorhersagen werden ebenfalls in diesen Ordner geschrieben.

In [1]:
# # Nur für Google Colab:
# # Hochladen von input.xlsx und final_model.pkl im Dialog
# from google.colab import files
# data_to_load = files.upload()
# 
# % pip install scikit-learn==0.23.2

In [2]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import pickle
import zipfile

# Lese Datei mit bisherigen Beobachtungen
# Benennung analog zu vorheriger Arbeit
data = pd.read_excel(
    'input.xlsx',
    names=[
        'fdist_id',
        'year',
        'forest_ownership',
        'timeframe',
        'RRK',
        'TM0',
        'demolition_wood',
        'infested_wood'
    ]
)

# Einstellungen

Die Vorhersageparameter im folgenden Codeblock können vom Benutzer frei verändert werden und haben Einfluss auf die Berechnung der Vorhersagen. Der restliche Code kann dann direkt ausgeführt werden, ohne die Notwendigkeit weiterer Anpassungen. Eine Ausnahme wäre beispielsweise, wenn eine Umstrukturierung der Forstreviere stattfindet. Dann wären die später zugeordneten gefährdeten Waldflächen anzupassen.

Die Vorhersagen werden für drei Szenarien getroffen, welche über je zwei Parameter definiert sind. Ein Parameter (s_kli) definiert die klimatischen Bedingungen und ein zweiter das anfallende Wurf- und Bruchholz (s_wbh). Die Parameter können getrennt voneinander definiert und mit beliebigen validen Werten bestückt werden, die Namen der Szenarien sind allerdings immer: 1 - warm/trocken, 2 - gemäßigt/durchschnittlich, 3 - kalt/feucht. 

Die Angabe der Parameter kann entweder als vierstellige Jahreszahl oder über die Angabe eines Quantils im Bereich 1-99 erfolgen. Kombinationen dieser zwei Möglichkeiten innerhalb eines Szenarios sind erlaubt. Wird eine Jahreszahl (z.B: 2015) gewählt, dann werden die klimatischen bzw. Bruchholz-Parameter aus diesem Jahr referenziert. Bei Angabe eines Quantils wird das jeweilige Quantil aus der gesamten Historie in "input.xlsx" für den jeweiligen Monat berechnet und übernommen. Ein höherer Wert des Quantils des klimatischen Parameters drückt aus, dass das Klima trockener und wärmer wird (d.h. bei 75 wird das 75-%-Quantil der Temperatur und das 25-%-Quantil des Niederschlags berechnet). Das 50-%-Quantil entspricht dem Median.

Ein letzer Einstellungswert gibt das Ende des Vorhersagezeitraums an. Ist die letzte reale Beobachtung in "input.xlsx" beispielsweise der September 2020 und dieser Parameter wird als "2021-05" definiert, dann werden aufeinanderfolgend der Zeitraum "Oktober-Dezember", "Januar-März", "April" und "Mai" berechnet, also alles zwischen dem letzten realen Eintrag bis einschließlich dem Ende des angegebenen Monats. Prinzipiell wird empfohlen, keine Vorhersagen für die ferne Zukunft zu treffen, denn mehrere Prädiktoren werden aus dem vergangenen Zugang von Schadholz berechnet. Da das Modell immer wieder seinen eigenen Output (Vorhersage) als Input (Beobachtung) für die nächste Periode nimmt und keine Begrenzung der Rückkopplung implementiert ist, können sich Abweichungen sowie falsche Annahmen des Modells potenzieren. Auch aus diesem Grund ist es wichtig, dass reale, aktuelle Beobachtungen in die input-Datei eingepflegt werden, um Korrekturen der Vorhersagen seitens des Modells zu ermöglichen.

In [3]:
# Vorhersageparameter
# Szenarien für Wetter und Wurf-/Bruchholz

# Szenario warm/trocken
s_kli_1 = 2018
s_wbh_1 = 2018

# Szenario gemäßigt/durchschnittlich
s_kli_2 = 50
s_wbh_2 = 50

# Szenario kalt/feucht
s_kli_3 = 2010
s_wbh_3 = 2010


# Ende Vorhersagezeitraum ('YYYY-MM')
# Monate 03-09 sowie 12 erlaubt
pred_end = '2022-03'

# Überprüfung des Inputs

Im nachfolgenden Code werden die eingelesene Excel-Datei "input.xlsx" sowie die angegebenen Parameter nach Fehlern und Problemen durchsucht. Eventuelle Warnungen werden nachfolgend ausgegeben.

In [4]:
# Überprüfe Integrität der Daten

# Zähle Anzahl Warnungen mit counter
warn_count = 0

################################################################################

# Keine leeren Einträge?
n_nan = data.isna().sum().sum()

if n_nan == 0:
    # Keine leeren Einträge in Beobachtungen
    pass
else:
    print(
        f'Warnung: {n_nan} leere Einträge in bisherigen Beobachtungen.\n'
        f'Dies beeinträchtigt unter Umständen die Qualität der Vorhersagen.\n'
    )
    warn_count += 1
    
################################################################################

# Keine Duplikate?
dup_rows = data.duplicated(
    ['fdist_id', 'forest_ownership', 'year', 'timeframe']
)

if dup_rows.any()==False:
    # Keine doppelten Einträge im Datansatz
    pass
else:    
    print(
        f'Warnung: Mehrere Einträge bestimmter Beobachtungen gefunden.\n'
        f'Folgende Zeilen werden gelöscht: {data[dup_rows]}.\n'
    )
    
    warn_count += 1
    
    data.drop_duplicates(
        ['fdist_id', 'forest_ownership', 'year', 'timeframe'], 
        inplace=True
    )

################################################################################

# Überprüfe, inwiefern aktuelle Werte für alle Reviere vorliegen

# Aktuellen Zeitraum bestimmen
max_year = data['year'].max()
max_timeframe = data[data['year'] == max_year]['timeframe'].max()

newest_data = data.loc[
    (data['year'] == max_year) & (data['timeframe'] == max_timeframe)
]

if newest_data.shape[0] == 106:
    # Aktuelle Werte für alle Forstreviere gefunden
    newest_ids = newest_data['fdist_id'].unique()
else:
    # Vorhandene IDs
    newest_ids = newest_data['fdist_id'].unique()
    
    # Vorhandene IDs mit nur einer Eigentumsgruppe
    mo_ids = [
        ID for ID in newest_ids 
        if newest_data[newest_data['fdist_id']==ID].shape[0] == 1
    ]
    
    # Ausgabe
    print(
        f'Warnung: Aktuelle Werte für {len(newest_ids)}' 
        f'Forstreviere gefunden.\n'
        f'In {len(mo_ids)} dieser Reviere fehlt eine Eigentumsgruppe.\n'
        f'Nur vollständige Reviere werden nachfolgend einbezogen.\n'
    )
    
    warn_count += 1
    
    # nur mit vollständigen Revieren weiterrechnen
    newest_ids = [x for x in newest_ids if x not in mo_ids]

################################################################################ 

# Überprüfe, ob für alle Reviere die 7 letzten Beobachtungen existieren  

def eotf(x):
    '''
    Berechne Ende eines Zeitraums. 
    input:
        - x: ein timestamp (datetime object)
    returns:
        - Ende des Monats (April-September) oder Ende des übernächsten
          Monats (Januar, Oktober) als datetime object
    '''

    if x.month in range(4, 10):
        return x + MonthEnd()
    else:
        return x + MonthEnd(3)
    

for ID in newest_ids:
    for fo in ['NSW', 'SW']:

        sub = data[
            (data['fdist_id'] == ID) & (data['forest_ownership'] == fo)
        ].copy()
        
        sub['ts'] = sub['year'].astype(str) + sub['timeframe'].map(
            lambda x: '-' + x.split(' ')[0])
        sub['ts'] = pd.to_datetime(sub['ts']).map(lambda x: eotf(x))
        
        max_ts = str(max_year) + '-' + max_timeframe.split(' ')[0]
        max_ts = eotf(pd.to_datetime(max_ts))
        cur_ts = max_ts
        
        for i in range(1,8):
            if cur_ts.month in range(4, 10):
                cur_ts = cur_ts + MonthEnd(-1)
            elif cur_ts.month in (3, 12):
                cur_ts = cur_ts + MonthEnd(-3)
            else:
                print(
                    f'Fehler: Ungültige Monate in Beobachtungen.\n'
                    f'Revier {ID}, Eigentumsgr. {fo}, Periode {cur_ts}.\n' 
                    f'Perioden beginnend mit 02, 03, 11, 12 nicht erlaubt.\n'
                    f'Die Ausführung des Codes wird unterbrochen.'
                )
                raise SystemExit(f'Ungültige Periode {ID}, {fo}, {cur_ts}!')
                
            if (sub['ts'] == cur_ts).any():
                pass
            else:
                print(
                    f'Fehler: Benötigter Eintrag nicht gefunden.\n'
                    f'Revier {ID}, Eigentumsgr. {fo}, Periode {cur_ts}.\n'
                    f'Alle Forstreviere, für die Vorherzusagen zu treffen '
                    f'sind, benötigen Beobachtungen für das letzte Jahr.'
                )
                raise SystemExit(f'Fehlender Eintrag {ID}, {fo}, {cur_ts}!')

# Alle benötigten historischen Einträge gefunden

################################################################################

# Überprüfung der Vorhersageparameter
for p in [s_kli_1, s_wbh_1, s_kli_2, s_wbh_2, s_kli_3, s_wbh_3]:
    if (p in range(1,100)) or (p in data['year'].unique()):
        pass
    else:
        raise SystemExit(
            'Ungültige Vorhersageparameter. Die Szenarienparameter müssen '
            'zwischen 1 und 99 liegen oder einer im Datenset enthaltenen '
            'Jahreszahl entsprechen.'
        )
        
pred_end = pd.to_datetime(pred_end) + MonthEnd()   

if pred_end - max_ts < pd.Timedelta('0'):
    raise SystemExit(
        'Ungültige Vorhersageparameter. Das Ende des Vorhersagezeitraums '
        'kann nicht in der Vergangenheit liegen.'
        )
elif pred_end - max_ts > pd.Timedelta('470 days'):
    print(
        'Warnung: Vorhersagen für die ferne Zukunft werden nicht empfohlen.'
    )
    warn_count += 1

################################################################################  

print(
    f'Überprüfung des Datensets und der Vorhersageparameter abgeschlossen.\n'
    f'Es traten 0 Fehler und {warn_count} Warnung(en) auf. '
)    



Warnung: 4 leere Einträge in bisherigen Beobachtungen.
Dies beeinträchtigt unter Umständen die Qualität der Vorhersagen.

Warnung: Vorhersagen für die ferne Zukunft werden nicht empfohlen.
Überprüfung des Datensets und der Vorhersageparameter abgeschlossen.
Es traten 0 Fehler und 2 Warnung(en) auf. 


# Vorbereitung der Vorhersage

Der nachfolgende Codeblock enthält Funktionen zum Erstellen der für die Vorhersage benötigten Prädiktoren aus "input.xlsx". Die Zuordnung von Informationen zu Forstrevieren, Zeiträumen etc. erfolgt über verschiedene Dictionaries. Dies hat den Vorteil, dass im Falle von Umstrukturiereungen etc. die Werte durch den Benutzer sehr einfach "händisch" angepasst werden können. Weterhin müssen diese Information so nicht aus zusätzlichen Daten in Google Colab eingelesen werden. Nachteil dieser Vorgehensweise ist, dass der Code dadurch an Länge gewinnt und eventuell unübersichtlicher wird.

In [5]:
# Dictionary für die Zuordnung der (neuen) Namen der Forstreviere
# Wie auch in den vorherigen Notebooks des Projekts wird für die "alten" 
# Reviere mit Führungsneun der Name des aktuellen Reviers verwendet, der dieses 
# am besten approximiert. 
# Diese logische Verknüpfung ist notwendig, damit das richtige Revier gefunden
# wird, wenn als Szenario ein Jahr vor 2015 verwendet wird.
fdist_names = {
    2501: 'Elsterheide',
    2502: 'Bernsdorf',
    2503: 'Königswartha',
    2504: 'Nebelschütz',
    2505: 'Königsbrück',
    2506: 'Radibor',
    2507: 'Kamenz',
    2508: 'Ohorn',
    2509: 'Bischofswerda',
    2510: 'Cunewalde',
    1101: 'Chemnitz',
    1201: 'Dresden',
    2101: 'Eibenstock',
    2102: 'Zwönitz',
    2103: 'Stollberg',
    2104: 'Zschopau',
    2105: 'Annaberg',
    2106: 'Marienberg',
    2107: 'Olbernhau',
    2191: 'Eibenstock',
    2192: 'Schwarzenberg',
    2193: 'Zwönitz',
    2194: 'Stollberg',
    2195: 'Annaberg',
    2196: 'Zschopau',
    2197: 'Marienberg',
    2198: 'Olbernhau',
    2201: 'Geringswalde',
    2202: 'Striegistal',
    2203: 'Reinsberg',
    2204: 'Frauenstein',
    2601: 'Zittau',
    2602: 'Löbau',
    2603: 'Niesky',
    2604: 'Krauschwitz',
    2605: 'Boxberg',
    2606: 'Weißwasser',
    2901: 'Muldental',
    2902: 'Leipziger Land',
    2701: 'M Nord',
    2702: 'M Ost',
    2703: 'M Süd',
    2704: 'M West',
    2791: 'M Nord',
    2792: 'M Süd',
    2793: 'M Ost',
    2801: 'Freital',
    2802: 'Glashütte',
    2803: 'Bad-Gottleuba',
    2804: 'Pirna',
    2805: 'Sebnitz',
    3001: 'Delitzsch',
    3002: 'Torgau',
    3003: 'Oschatz',
    2301: 'Adorf',
    2302: 'Schöneck',
    2303: 'Weischlitz',
    2304: 'Plauen',
    2305: 'Treuen',
    2306: 'Auerbach',
    2401: 'Z Nord',
    2402: 'Z Süd',
    1302: 'Connewitz',
    1301: 'Leutzsch'
}

# Gefährdete Waldfläche (SW) aus der Nummer des Forstreviers
endarea_sw = {
    2501: 26.85,
    2502: 82.46,
    2503: 24.17,
    2504: 8.68,
    2505: 483.53,
    2506: 63.43,
    2507: 3.43,
    2508: 1417.36,
    2509: 127.91,
    2510: 8.3,
    1101: 774.06,
    1201: 1762.71,
    2101: 15177.9,
    2102: 2847.32,
    2103: 1921.76,
    2104: 3646.39,
    2105: 7210.79,
    2106: 3946.53,
    2107: 2314.46,
    2191: 10922.47,
    2192: 6940.4,
    2193: 1290.14,
    2194: 2364.83,
    2195: 6142.78,
    2196: 3191.06,
    2197: 3892.5,
    2198: 2315.36,
    2201: 196.15,
    2202: 1147.04,
    2203: 2706.18,
    2204: 3833.76,
    2601: 14.92,
    2602: 20.78,
    2603: 8.2,
    2604: 122.78,
    2605: 71.88,
    2606: 42.73,
    2901: 51.06,
    2902: 615.51,
    2701: 3.85,
    2702: 0.6,
    2703: 381.91,
    2704: 0.08,
    2791: 3.93,
    2792: 381.83,
    2793: 1.09,
    2801: 5291.51,
    2802: 4507.81,
    2803: 2690.42,
    2804: 1584.51,
    2805: 8458.97,
    3001: 0.12,
    3002: 0.0,
    3003: 465.65,
    2301: 4312.15,
    2302: 8310.18,
    2303: 791.57,
    2304: 779.57,
    2305: 475.83,
    2306: 2924.21,
    2401: 1348.61,
    2402: 196.48,
    1302: 0.0,
    1301: 0.0
}

# Gefährdete Waldfläche (NSW) aus der Nummer des Forstreviers
endarea_nsw = {
    2501: 11.37,
    2502: 60.79,
    2503: 109.44,
    2504: 231.78,
    2505: 173.07,
    2506: 246.66,
    2507: 721.84,
    2508: 1028.08,
    2509: 2184.61,
    2510: 3323.88,
    1101: 280.84,
    1201: 124.17,
    2101: 929.71,
    2102: 2953.21,
    2103: 2903.46,
    2104: 1745.57,
    2105: 1855.4,
    2106: 2062.88,
    2107: 1379.08,
    2191: 726.57,
    2192: 1143.21,
    2193: 3482.08,
    2194: 1898.64,
    2195: 2408.13,
    2196: 1577.87,
    2197: 1208.74,
    2198: 1380.03,
    2201: 841.61,
    2202: 954.18,
    2203: 1597.32,
    2204: 1708.41,
    2601: 3757.67,
    2602: 2858.26,
    2603: 631.21,
    2604: 401.0,
    2605: 146.13,
    2606: 103.7,
    2901: 225.27,
    2902: 401.71,
    2701: 33.41,
    2702: 114.56,
    2703: 392.75,
    2704: 36.08,
    2791: 22.8,
    2792: 411.13,
    2793: 143.31,
    2801: 1675.64,
    2802: 1439.74,
    2803: 1073.94,
    2804: 1153.3,
    2805: 2152.83,
    3001: 11.28,
    3002: 14.46,
    3003: 271.09,
    2301: 3922.55,
    2302: 2342.52,
    2303: 2123.89,
    2304: 2152.93,
    2305: 2621.37,
    2306: 2748.21,
    2401: 1319.45,
    2402: 1794.78,
    1302: 0.54,
    1301: 0.0
}

# Numerische codierung des Zeitraums analog zu Trainings-Notebook
timeframe_encoder = {
    '01 Januar-März': 0.04695437961628093, 
    '04 April': 0.01613466392797747, 
    '05 Mai': 0.026238326902676162, 
    '06 Juni': 0.048392631582756127, 
    '07 Juli': 0.11820248134015453, 
    '08 August': 0.2340321842232208, 
    '09 September': 0.2396741387722562, 
    '10 Oktober-Dezember': 0.2703711936346777
}

# Name des Zeitraums in "Sachsenforst-Notation" aus Nummer des letzten Monats
tf_from_month = {
    3:  '01 Januar-März',
    4:  '04 April',
    5:  '05 Mai',
    6:  '06 Juni',
    7:  '07 Juli',
    8:  '08 August',
    9:  '09 September',
    12: '10 Oktober-Dezember'
}
    
# Erstelle Spalte mit (neuem) Namen der Forstreviere
data['fdist_newname'] = data['fdist_id'].map(lambda x: fdist_names.get(x))

# Funktion, um 'Skelett' für nächste Vorhersage zu erstellen
# d.h. Dataframe nur mit deskriptiven Spalten
def prepare_next(
    newest_ids, cur_ts, 
    fdist_names=fdist_names, 
    tf_from_month=tf_from_month
):
    '''
    Diese Funktion erstellt einen neuen Dataframe für die nächste Vorhersage.
    Der Dataframe erhält die Spalten für das Forstrevier (id, name), 
    die Eigentumsgruppe, das Jahr sowie den Zeitraum.
    
    input:
        - newest_ids: Alle in der neuen Vorhersage enthaltenen REVUFBADR-Nummern
        - cur_ts: Zeitraum der neuen Vorhersage als datetime-object
        - fdist_names: Dictionary mit Zuordnung der Forstrevier-Namen
        - tf_from_month: Dictionary mit Zuordnung der kategorischen Zeiträume
        
    returns:
        - Dataframe als Grundlage für nächste Vorhersage
    '''  
    
    # Erstellung der späteren Spalten als numpy arrays
    # Jede REVUFBADR (fdist_id) zweimal, da zwei Eigentumsgruppen
    fdist_id = np.append(newest_ids, newest_ids)
    
    # Spalte mit Jahr in gleicher Länge
    year = [cur_ts.year] * 2 * len(newest_ids)
    
    # Eigentumsgruppe je einmal für die Forstreviere
    forest_ownership = np.append(
        ['NSW'] * len(newest_ids), 
        ['SW'] * len(newest_ids)
    )
    
    # Zeitraum als Monat, aus cur_ts
    timeframe = [cur_ts.month]* 2 * len(newest_ids)
    # Zeitraum zurücksetzen in Notierung von Sachsenforst über Dictionary
    timeframe = np.array([tf_from_month.get(x) for x in timeframe])
    
    # Forstrevier Name über Dictionary
    fdist_newname = [fdist_names.get(x) for x in fdist_id]
    
    # Zusammenführen der Spalten in Dataframe
    df = pd.DataFrame([
        fdist_id, 
        year, 
        forest_ownership, 
        timeframe, 
        fdist_newname
    ]).T
    
    # Spaltennamen festlegen
    df.columns =[
        'fdist_id',
        'year',
        'forest_ownership',
        'timeframe',
        'fdist_newname'
    ]
    
    return df
    
    
    
# Funktion, um 'Skelett' mit Features zu füllen
def populate_features(
    X, data, original_data, s_kli, s_wbh, cur_ts,
    endarea_sw=endarea_sw, 
    endarea_nsw=endarea_nsw, 
    tf_from_month=tf_from_month,
    timeframe_encoder=timeframe_encoder
):
    '''
    Diese Funktion nimmt einen Dataframe mit deskriptiven Spalten und
    ergänzt alle für die Vorhersage benötigten Features, indem es aus den
    historischen Beobachtungen und den letzten Beobachtungen/Vorhersagen die
    jeweiligen Werte heraussucht und diese aggregiert bzw. verarbeitet.
    
    inputs:
        - X: Der Dataframe, an den die Features angehangen werden und für 
          den später die Vorhersage stattfindet
        - data: Dataframe mit allen bisherigen realen Beobachtungen sowie 
          den bisherigen Vorhersagen. Relevant, wenn das jüngste Schadholz
          einbezogen wird
        - original_data: Dataframe nur mit realen Beobachtungen. Relevant, 
          wenn Klima bestimmt wird, da nur reale Beobachtungen Grundlage für
          Quantil sind
        - s_kli: Aktuelles Klimaszenario
        - s_wbh: Szenario für Wurf- und Bruchholz
        - cur_ts: aktuelle Periode als datetime-object
        - endarea_sw: Dictionary mit Zuordnung der gefährdeten Waldfläche der 
          Forstreviere für Staatswald
        - endarea_nsw: Dictionary mit Zuordnung der gefährdeten Waldfläche der 
          Forstreviere für Nichtstaatswald
        - tf_from_month: Dictionary mit Zuordnung der kategorischen Zeiträume
        - timeframe_encoder; Dictionary mit Zuordnung der Numerischen Faktoren
          zu den Zeiträumen, analog des Modelltrainings
          
    returns:
        - DataFrame mit allen benötigten Zeilen und Spalten für die Vorhersage 
          des nächsten Zeitraums mit dem trainierten Modell
    '''
    ############################################################################

    # Vorbereitung: 'data' kopieren (original sollte unver#ndert bleiben) 
    df = data.copy()
    
    # Vorbereitung: Spalte für timestamp ergänzen
    df['ts'] = df['year'].astype(str) + df['timeframe'].map(
            lambda x: '-' + x.split(' ')[0])
    df['ts'] = pd.to_datetime(df['ts']).map(lambda x: eotf(x))
    
    # Vorbereitung: Vorheriger timestamp
    if cur_ts.month in range(4,10):
        prev_ts = cur_ts + MonthEnd(-1)
    else:
        prev_ts = cur_ts + MonthEnd(-3)
        
    ############################################################################ 

    # Feature: area_endangered
    X['area_endangered'] = X[['fdist_id','forest_ownership']].apply(
        lambda x: endarea_sw.get(x[0]) if x[1]=='SW' else endarea_nsw.get(x[0]),
        axis=1
    )
    
    ############################################################################    

    # Features zu bisherigem Schadholz
    prev_inf_wood = []
    prev_inf_wood_ofo = []
    prev_infested_Wood_rollyr = []
    
    for r in X[['fdist_newname', 'forest_ownership']].itertuples(index=False):
        # Feature: prev_infested_wood
        # Wert für letzten Zeitraum des gleichen Reviers und Eigentumgruppe
        piw = df.loc[
            (df['fdist_newname'] == r[0]) &
            (df['forest_ownership'] == r[1]) &
            (df['ts'] == prev_ts)
        ]['infested_wood'].values[0]
        
        prev_inf_wood.append(piw)
        
        # Feature: prev_infested_wood_ofo
        
        piwo = df.loc[
            (df['fdist_newname'] == r[0]) &
            (df['forest_ownership'] != r[1]) &
            (df['ts'] == prev_ts)
        ]['infested_wood'].values[0]
        
        prev_inf_wood_ofo.append(piwo)
        
        # Feature: prev_infested_Wood_rollyr
        # Summe der letzten Jahreswerte für jew. Revier und Eigentumsgruppe
        piwryr = df.loc[
            (df['fdist_newname'] == r[0]) &
            (df['forest_ownership'] == r[1]) &
            (df['ts'] >= cur_ts + MonthEnd(-12)) & 
            (df['ts'] < cur_ts)
        ]['infested_wood'].sum()
        
        prev_infested_Wood_rollyr.append(piwryr)
        
    X['prev_infested_wood'] = prev_inf_wood
    X['prev_infested_wood_ofo'] = prev_inf_wood_ofo
    X['prev_infested_wood_rollyr'] = prev_infested_Wood_rollyr
        
    ############################################################################ 

    # Klimatische Features
    rrk = []
    tm0 = []
    rrk_rollsr = []
    
    # Möglichkeit 1: Szenario als Jahreszahl ausgewählt
    if s_kli in original_data['year'].unique():
        for r in X[[
            'fdist_newname', 'forest_ownership', 'timeframe'
        ]].itertuples(index=False):
            # Problem wenn Jahreszahl <= 2013 und fdist_newname 'Meißen West' 
            # Eintrag existiert nicht. Workaround in folgendem if-statement:
            if r[0] == 'M West' and s_kli <= 2013:
                # Feature: RRK
                rrk_s = original_data.loc[
                    (original_data['fdist_newname'] == 'M Nord') &
                    (original_data['forest_ownership'] == r[1]) &
                    (original_data['timeframe'] == r[2]) &    
                    (original_data['year'] == s_kli)  
                ]['RRK'].values[0]
                
                #Feature: TM0
                tm0_s = original_data.loc[
                    (original_data['fdist_newname'] == 'M Nord') &
                    (original_data['forest_ownership'] == r[1]) &
                    (original_data['timeframe'] == r[2]) &    
                    (original_data['year'] == s_kli)  
                ]['TM0'].values[0]
                
            # Für die anderen Reviere wie folgt:    
            else:
                # Feature: RRK
                rrk_s = original_data.loc[
                    (original_data['fdist_newname'] == r[0]) &
                    (original_data['forest_ownership'] == r[1]) &
                    (original_data['timeframe'] == r[2]) &    
                    (original_data['year'] == s_kli)  
                ]['RRK'].values[0]
                
                #Feature: TM0
                tm0_s = original_data.loc[
                    (original_data['fdist_newname'] == r[0]) &
                    (original_data['forest_ownership'] == r[1]) &
                    (original_data['timeframe'] == r[2]) &    
                    (original_data['year'] == s_kli)  
                ]['TM0'].values[0]
                
            rrk.append(rrk_s)
            tm0.append(tm0_s)
            
            # Feature: RRK_rollsr
            # Summe der letzten (inklusive aktueller) Sommerwerte des 
            # letzten Jahres aus jew. Revier und Eigentumsgruppe
            # letze Werte
            rrk_rollsr1 = df.loc[
                (df['fdist_newname'] == r[0]) &
                (df['forest_ownership'] == r[1]) &
                (df['ts'] >= cur_ts + MonthEnd(-11)) & 
                (df['ts'] <= cur_ts) &
                (df['ts'].map(lambda x: x.month).isin(range(4,10)))
            ]['RRK'].sum()
            
            # aktueller Wert
            rrk_rollsr2 = rrk_s
            
            rrk_rollsr.append((rrk_rollsr1 + rrk_rollsr2) / 6)
                
    # Möglichkeit 2: Szenario als Quantil ausgewählt        
    elif s_kli in range(1,100):
        for r in X[[
            'fdist_newname', 'forest_ownership', 'timeframe'
        ]].itertuples(index=False):
            # Feature: RRK
            rrk_s = np.quantile(
                original_data.loc[
                    (original_data['fdist_newname'] == r[0]) &
                    (original_data['forest_ownership'] == r[1]) &
                    (original_data['timeframe'] == r[2])  
                ]['RRK'].values, 
                1 - (s_kli * 0.01)
            )
            
            rrk.append(rrk_s)
            
            # Feature: TM0
            tm0_s = np.quantile(
                original_data.loc[
                    (original_data['fdist_newname'] == r[0]) &
                    (original_data['forest_ownership'] == r[1]) &
                    (original_data['timeframe'] == r[2])  
                ]['TM0'].values, 
                s_kli * 0.01
            )
            
            tm0.append(tm0_s)
            
            # Feature: RRK_rollsr
            # Summe der letzten (inklusive aktueller) Sommerwerte des 
            # letzten Jahres aus jew. Revier und Eigentumsgruppe
            # letze Werte
            rrk_rollsr1 = df.loc[
                (df['fdist_newname'] == r[0]) &
                (df['forest_ownership'] == r[1]) &
                (df['ts'] >= cur_ts + MonthEnd(-11)) & 
                (df['ts'] <= cur_ts) &
                (df['ts'].map(lambda x: x.month).isin(range(4,10)))
            ]['RRK'].sum()
            
            # aktueller Wert
            rrk_rollsr2 = rrk_s
            
            rrk_rollsr.append((rrk_rollsr1 + rrk_rollsr2) / 6)

            
    X['RRK'] = rrk
    X['TM0'] = tm0
    X['RRK_rollsr'] = rrk_rollsr
    ############################################################################ 

    # Wurf-/Bruchholz
    dmw = []
    
    # Möglichkeit 1: Szenario als Jahreszahl ausgewählt
    if s_wbh in original_data['year'].unique():
        for r in X[[
            'fdist_newname', 'forest_ownership', 'timeframe'
        ]].itertuples(index=False):
            # Feature: demolition_wood
            # Problem wenn Jahreszahl <= 2013 und fdist_newname 'Meißen West' 
            # Eintrag existiert nicht. Workaround in folgendem if-statement
            if r[0] == 'M West' and s_kli <= 2013:
                dmw_s = original_data.loc[
                    (original_data['fdist_newname'] == 'M Nord') &
                    (original_data['forest_ownership'] == r[1]) &
                    (original_data['timeframe'] == r[2]) &    
                    (original_data['year'] == s_wbh)  
                ]['demolition_wood'].values[0]
            
            # Für die anderen Reviere wie folgt: 
            else:
                dmw_s = original_data.loc[
                    (original_data['fdist_newname'] == r[0]) &
                    (original_data['forest_ownership'] == r[1]) &
                    (original_data['timeframe'] == r[2]) &    
                    (original_data['year'] == s_wbh)  
                ]['demolition_wood'].values[0]
            
            dmw.append(dmw_s)
            
    # Möglichkeit 2: Szenario als Quantil ausgewählt          
    elif s_wbh in range(1,100):
        for r in X[[
            'fdist_newname', 'forest_ownership', 'timeframe'
        ]].itertuples(index=False):
            # Feature: demolition_wood
            dmw_s = np.quantile(
                original_data.loc[
                    (original_data['fdist_newname'] == r[0]) &
                    (original_data['forest_ownership'] == r[1]) &
                    (original_data['timeframe'] == r[2])  
                ]['demolition_wood'].values, 
                s_wbh * 0.01
            )
            
            dmw.append(dmw_s)
            
    X['demolition_wood'] = dmw
            
    ############################################################################ 
    
    # Feature: timeframe (encoded)
    X['timeframe_enc'] = X['timeframe'].map(lambda x: timeframe_encoder.get(x))
    
    
    return X    
        
# Lade Modell
model = pickle.load(open('model.pkl', 'rb'))    

# Vorhersagen für Szenarien durchführen

Nachdem alle benötigten Variablen und Funktionen definiert wurden, kann die Vorhersage für die drei Szenarien durchgeführt werden.

In [ ]:
# Ursprüngliche Werte der Daten festhalten 
original_data = data.copy()

# Vorhersagen für drei Szenarien
for i in range(3):
    # Vor dem Szenario 'data' zurücksetzen
    data = original_data.copy()
    
    # Zähler für Intervalle erstellen
    period_count = 0
    pred_range = pd.date_range(start=max_ts, end=pred_end, freq='M')
    max_periods = len(
        [ts for ts in pred_range if ts.month not in (10,11,1,2)]
    ) - 1
    
    # Szenario über Einstellungen definieren
    if i == 0:
        s_name = 'warmtrocken'
        s_kli = s_kli_1
        s_wbh = s_wbh_1
        print('Starte Vorhersage für Szenario warm/trocken.')
    elif i == 1:
        s_name = 'gemäßigt'
        s_kli = s_kli_2
        s_wbh = s_wbh_2
        print('Starte Vorhersage für Szenario gemäßigt.')
    elif i == 2:
        s_name = 'kaltfeucht'
        s_kli = s_kli_3
        s_wbh = s_wbh_3
        print('Starte Vorhersage für Szenario kalt/feucht.')
        
    cur_ts = max_ts     
    
    # Algorithmus ausführen, bis Ende Vorhersagezeitraum erreicht
    while cur_ts < pred_end:
        
        # Eine Periode nach vorne
        period_count += 1
        
        if cur_ts.month in range(3,9):
            cur_ts = cur_ts + MonthEnd(1)
        else:
            cur_ts = cur_ts + MonthEnd(3)
        
        # Dataframe kreieren und für Vorhersage mit Features füllen
        X = prepare_next(newest_ids, cur_ts)
        X = populate_features(X, data, original_data, s_kli, s_wbh, cur_ts)
        
        # Vorhersage durchführen
        X['infested_wood'] = model.predict(X[[
            'area_endangered',
            'timeframe_enc',
            'prev_infested_wood',
            'prev_infested_wood_rollyr',
            'prev_infested_wood_ofo',
            'RRK',
            'TM0',
            'demolition_wood',
            'RRK_rollsr'
        ]])
        
        # Vorhersagen und Datenset vereinigen als Vorbereitung 
        # für nächste Iteration
        data = pd.concat(
            [data, 
            X[[
                'fdist_id', 
                'year', 
                'forest_ownership', 
                'timeframe', 
                'RRK', 
                'TM0', 
                'demolition_wood', 
                'infested_wood',
                'fdist_newname'
            ]]], 
            ignore_index=True
        )
        
        print(f'Intervall {period_count}/{max_periods} abgeschlossen')
    
    # Nachdem alle Vorhersagen für Szenario fertig, Vohersagen aus data extrahieren
    predictions = data.loc[original_data.shape[0] + 1 :]

    predictions.columns = [
        'REVUFBADR',
        'Jahr',
        'Eigentumsgruppe',
        'ZR',
        'Niederschlagsumme in l/m2',
        'Mittlere Temperatur in °C',
        'Zugang Wurf-/Bruchholz',
        'Zugang Schadholz',
        'Revier'
    ]
    
    # Speichern der 'ausführlichen' Version der Vorhersagen in Excel-Datei
    predictions.to_excel(
        'vorhersagen_ausführlich_'+s_name+'.xlsx',
        index=False
    )
    
    # Aufsummierung der Ergebnisse für die einzelnen Reviere, für Gesamtvorhersage
    predictions = predictions.groupby(
        ['REVUFBADR', 'Revier', 'Eigentumsgruppe']
    )['Zugang Schadholz'].sum()
    
    # Speichern der aggregierten Ergebnisse in weiterer Excel-Datei
    predictions.to_excel(
        'vorhersagen_gesamt_'+s_name+'.xlsx'
    )
    
    print(
        f'Vorhersage für Szenario abgeschlossen. ' 
        f'Gespeichert in "vorhersagen_ausführlich_{s_name}.xlsx" '
        f'sowie "vorhersagen_gesamt_{s_name}.xlsx".\n'
    )
    
    

Starte Vorhersage für Szenario warm/trocken.
Intervall 1/10 abgeschlossen
Intervall 2/10 abgeschlossen
Intervall 3/10 abgeschlossen
Intervall 4/10 abgeschlossen
Intervall 5/10 abgeschlossen
Intervall 6/10 abgeschlossen
Intervall 7/10 abgeschlossen
Intervall 8/10 abgeschlossen
Intervall 9/10 abgeschlossen
Intervall 10/10 abgeschlossen
Vorhersage für Szenario abgeschlossen. Gespeichert in "vorhersagen_ausführlich_warmtrocken.xlsx" sowie "vorhersagen_gesamt_warmtrocken.xlsx".

Starte Vorhersage für Szenario gemäßigt.
Intervall 1/10 abgeschlossen
Intervall 2/10 abgeschlossen
Intervall 3/10 abgeschlossen
Intervall 4/10 abgeschlossen
Intervall 5/10 abgeschlossen
Intervall 6/10 abgeschlossen
Intervall 7/10 abgeschlossen
Intervall 8/10 abgeschlossen
Intervall 9/10 abgeschlossen
Intervall 10/10 abgeschlossen
Vorhersage für Szenario abgeschlossen. Gespeichert in "vorhersagen_ausführlich_gemäßigt.xlsx" sowie "vorhersagen_gesamt_gemäßigt.xlsx".

Starte Vorhersage für Szenario kalt/feucht.
Interval

Der folgende Code lädt die Excel-Dateien mit den Vorhersagen aus Google Colab herunter. Wird nicht benötigt, wenn Notebook lokal ausgeführt wird.

In [ ]:
# # Nur für Google Colab:
# # Dateien zu zip-Archiv hinzufügen
# zipfile = zipfile.ZipFile(
#     'vorhersagen.zip', 
#     mode='w', 
#     compression=zipfile.ZIP_DEFLATED
#     )
# 
# zipfile.write('vorhersagen_ausführlich_warmtrocken.xlsx')
# zipfile.write('vorhersagen_ausführlich_gemäßigt.xlsx')
# zipfile.write('vorhersagen_ausführlich_kaltfeucht.xlsx')
# zipfile.write('vorhersagen_gesamt_warmtrocken.xlsx')
# zipfile.write('vorhersagen_gesamt_gemäßigt.xlsx')
# zipfile.write('vorhersagen_gesamt_kaltfeucht.xlsx')
#  
# zipfile.close()
# 
# # Excel Dateien aus Google Colab downloaden
# files.download('vorhersagen.zip')